In [29]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [54]:
train_files_name = "dataset_small/train"
test_file_name = "dataset_small/test"

In [42]:
def load_dataset(path):
    data = load_files(path)
    dog_files = np.array(data['filenames'])
    dog_targets = np_utils.to_categorical(np.array(data['target']), 10)
    return dog_files, dog_targets

In [55]:
# load train, test, and validation datasets
train_files, train_targets = load_dataset(train_files_name)
# valid_files, valid_targets = load_dataset()
test_files, test_targets = load_dataset(test_file_name)

dog_names = [item[20:-1] for item in sorted(glob("dataset_small/train/*/"))]

In [44]:
print(dog_names)

['000.pekinese', '001.borzoi', '002.bluetick', '003.boston_bull', '004.bedlington_terrier', '005.scottish_deerhound', '006.golden_retriever', '007.dingo', '008.shetland_sheepdog', '009.basenji']


In [45]:
from keras.preprocessing import image                  
from tqdm import tqdm


In [18]:
def path_to_tensor(img_path, size=224):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(size, size))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

In [19]:
def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [20]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

In [56]:
# pre-process the data for Keras
train_tensors = paths_to_tensor(train_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255




  0%|          | 0/870 [00:00<?, ?it/s]


  1%|▏         | 12/870 [00:00<00:07, 118.55it/s]


  3%|▎         | 25/870 [00:00<00:06, 120.75it/s]


  4%|▍         | 38/870 [00:00<00:06, 123.32it/s]


  6%|▌         | 53/870 [00:00<00:06, 129.07it/s]


  8%|▊         | 69/870 [00:00<00:05, 136.49it/s]


 10%|█         | 87/870 [00:00<00:05, 145.91it/s]


 12%|█▏        | 101/870 [00:00<00:05, 133.13it/s]


 13%|█▎        | 116/870 [00:00<00:05, 136.67it/s]


 15%|█▌        | 131/870 [00:00<00:05, 139.50it/s]


 17%|█▋        | 145/870 [00:01<00:05, 135.34it/s]


 19%|█▊        | 161/870 [00:01<00:05, 140.55it/s]


 20%|██        | 176/870 [00:01<00:06, 108.63it/s]


 22%|██▏       | 191/870 [00:01<00:05, 116.24it/s]


 24%|██▍       | 207/870 [00:01<00:05, 124.93it/s]


 26%|██▌       | 223/870 [00:01<00:04, 132.43it/s]


 28%|██▊       | 241/870 [00:01<00:04, 143.81it/s]


 30%|██▉       | 257/870 [00:01<00:04, 146.41it/s]


 31%|███▏      | 273/870 [00:01<00:03, 149.64it/s]


 33%|██

 35%|███▍      | 606/1739 [00:08<00:13, 81.70it/s]


 35%|███▌      | 615/1739 [00:08<00:13, 81.48it/s]


 36%|███▌      | 624/1739 [00:08<00:13, 81.03it/s]


 36%|███▋      | 633/1739 [00:09<00:14, 77.83it/s]


 37%|███▋      | 642/1739 [00:09<00:13, 78.85it/s]


 37%|███▋      | 651/1739 [00:09<00:14, 74.86it/s]


 38%|███▊      | 659/1739 [00:09<00:14, 74.66it/s]


 38%|███▊      | 667/1739 [00:09<00:14, 71.66it/s]


 39%|███▉      | 677/1739 [00:09<00:13, 76.83it/s]


 39%|███▉      | 685/1739 [00:09<00:15, 69.94it/s]


 40%|███▉      | 693/1739 [00:09<00:14, 70.62it/s]


 40%|████      | 701/1739 [00:09<00:14, 70.68it/s]


 41%|████      | 709/1739 [00:10<00:14, 70.37it/s]


 41%|████      | 717/1739 [00:10<00:14, 71.65it/s]


 42%|████▏     | 725/1739 [00:10<00:14, 71.15it/s]


 42%|████▏     | 733/1739 [00:10<00:13, 72.64it/s]


 43%|████▎     | 741/1739 [00:10<00:14, 70.94it/s]


 43%|████▎     | 749/1739 [00:10<00:14, 68.91it/s]


 43%|████▎     | 756/1739 [00:10<00:15, 65.50i

In [27]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.regularizers import l2

In [47]:
model = Sequential()

### TODO: Define your architecture.
drop1 = 0.25     # initial dropout rate
drop2 = 0.5      # secondary dropout rate
d_act = 'relu'   # default activation function
d_pad = 'same'   # default padding
reg = l2(1e-3)   # L2 reg

model = Sequential()
model.add(Conv2D(16, 2, input_shape=train_tensors.shape[1:], padding=d_pad, activation=d_act, kernel_regularizer=reg, name='block1_conv1'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(32, 2, padding=d_pad, activation=d_act, kernel_regularizer=reg, name='block2_conv1'))
model.add(MaxPooling2D(pool_size=2))
model.add(Conv2D(64, 2, padding=d_pad, activation=d_act, kernel_regularizer=reg, name='block3_conv1'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(drop1))
model.add(Flatten())
model.add(Dense(512, activation=d_act, kernel_regularizer=reg))
model.add(Dropout(drop2))
model.add(Dense(len(dog_names), activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 16)      208       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 112, 112, 16)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 32)      2080      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 56, 56, 32)        0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 64)        8256      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 28, 28, 64)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 28, 28, 64)        0         
__________

In [48]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

In [49]:
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint

In [51]:
### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 20

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit(train_tensors, train_targets, epochs=epochs, batch_size=20, verbose=1)

Epoch 1/20
870/870 [==============================] - 45s 52ms/step - loss: 1.0525 - acc: 0.9057
Epoch 2/20
870/870 [==============================] - 42s 49ms/step - loss: 0.9789 - acc: 0.9276
Epoch 3/20
870/870 [==============================] - 39s 45ms/step - loss: 0.9574 - acc: 0.9103
Epoch 4/20
870/870 [==============================] - 39s 45ms/step - loss: 0.9490 - acc: 0.9218
Epoch 5/20
870/870 [==============================] - 45s 52ms/step - loss: 0.8759 - acc: 0.9379
Epoch 6/20
870/870 [==============================] - 48s 55ms/step - loss: 0.8258 - acc: 0.9368
Epoch 7/20
870/870 [==============================] - 42s 48ms/step - loss: 0.8302 - acc: 0.9310
Epoch 8/20
870/870 [==============================] - 39s 45ms/step - loss: 0.8509 - acc: 0.9299
Epoch 9/20
870/870 [==============================] - 46s 53ms/step - loss: 0.7605 - acc: 0.9448
Epoch 10/20
870/870 [==============================] - 51s 58ms/step - loss: 0.7705 - acc: 0.9264
Epoch 11/20
870/870 [========

In [57]:
dog_breed_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

# report test accuracy
test_accuracy = 100*np.sum(np.array(dog_breed_predictions)==np.argmax(test_targets, axis=1))/len(dog_breed_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

Test accuracy: 4.3128%


In [77]:
p = model.predict(np.expand_dims(test_tensors[1], axis=0))

In [78]:
print(np.argmax(p))
print(np.argmax(test_targets[1]))

9
7
